In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
import keras

In [ ]:
base_model=InceptionV3(input_shape=(256,256,3), include_top=False)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
X = Flatten()(base_model.output)
X = Dense(units=3, activation='sigmoid')(X)

model = Model(base_model.input, X)

model.compile(optimizer='adam',loss=keras.losses.binary_crossentropy, metrics=['accuracy'])

#summary
model.summary()

**Pre-Process data using Generator**

In [ ]:
train_datagen = ImageDataGenerator(zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.4,
    channel_shift_range=0.0,
    fill_mode='nearest')
#/kaggle/input/military-aircraft-recognition-dataset/Annotations/Oriented Bounding Boxes/

In [ ]:
train_data=train_datagen.flow_from_directory("/kaggle/input/military-aircraft-recognition-dataset", target_size=(256,256), batch_size=36)

In [ ]:
train_data.class_indices

In [ ]:
t_img, label = train_data.next()

In [ ]:
t_img

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

mc_point = ModelCheckpoint("mymodel.h5", moniter="accuracy", verbose=1, save_best_only=True)

In [ ]:
es=EarlyStopping(monitor="loss",
                 min_delta=0.01,
                patience=5,
                verbose=1)

cbk = [mc_point,es]

*Fit the model*

In [ ]:
hist=model.fit_generator(train_data, steps_per_epoch=20, epochs=2, callbacks=cbk)

**Validate model**

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt

In [ ]:
path="/kaggle/input/military-aircraft-recognition-dataset/JPEGImages/1970.jpg"

In [ ]:
img = load_img(path,target_size=(256,256))
ig=img_to_array(img)
i=preprocess_input(ig)
input_arr = np.array([i])
pred=np.argmax(model.predict(input_arr))
plt.imshow(ig)
print("Prediction")
if pred==0:
    print("**Type 0**\n")
elif pred==1:
    print("**Type 1**\n")
else:
    print("**Type 2**\n")